# Solving Sudoku using AI

In this project, we're going to be solving sudoku using Artifical Intelligence. 

For now, we will be using Sudoku puzzles from [this page](https://www.websudoku.com). 



### 1. Reading pdf
First, we are going to read the pdf and create an image out of it.
The image will have the name of the file + 'image.jpg'

To get sudokus:
    1. Go to websudoku.com
    2. Click Print
    3. Print large
    4. Save it in a folder.
   

In [1]:
from wand.image import Image as WandImage

def pdf_to_image(file):
    with(WandImage(filename=file, resolution=120)) as source: 
        for i, image in enumerate(source.sequence):
            newfilename = file[:-4] + '_image' + '.jpeg'
        WandImage(image).save(filename=newfilename)
        
pdf_to_image('sudoku images/sudoku_directly_from_site.pdf')

### 2. Crop sudoku
Now, we are going to crop the sudoku so it's all we have on our image.

In [2]:
from PIL import Image,ImageOps 

def image_crop(image):
    border = (165, 410, 160, 240) # left, up, right, bottom  (250, 500, 250, 320)
    im = Image.open(image)
    new_image = ImageOps.crop(im, border)
    new_image.save('sudoku images/sudoku.jpg')
image_crop('sudoku images/sudoku_directly_from_site_image.jpeg')


### 3. Getting the numbers
Now that we have a good image of the sudoku, we are going to cut it into pieces. We will cut it into 81 images, each standing for one square. This will help us be able to use a model to transform that image into a string later.

In [3]:
#4 1 0 0 0 0 0 0 0
#6 0 2 0 0 0 0 5 8
#7 3 0 1 0 8 0 0 0

#0 0 0 2 4 0 0 1 6
#3 0 6 8 0 1 7 0 5
#8 2 0 0 6 5 0 0 0

#0 0 0 9 0 6 0 8 4
#1 6 0 0 0 0 3 0 2
#0 0 0 0 0 0 0 9 1
#695 x 670 
#77 pixels approx each square
#75 pixels approx each row  
#left, top, right, bottom 
def numbers_in_sudoku(image):
    im = Image.open(image) 
    numbers = [
        (25,20,635,615),(92,15,548,610),(169,15,471,610),(246,15,394,610),(323,15,317,610),(400,15,240,610),(477,15,163,610),(554,15,86,610),(631,15,9,610),
        (25,80,625,535),(92,80,548,535),(169,80,471,535),(246,80,394,535),(323,80,317,535),(400,80,240,535),(477,80,163,535),(554,80,86,535),(631,80,9,535),
        (25,160,625,455),(92,160,548,455),(169,160,471,455),(246,160,394,455),(323,160,317,455),(400,160,240,455),(477,160,163,455),(554,160,86,455),(631,160,9,455),

        (25,235,625,385),(92,235,548,380),(169,235,471,380),(246,235,394,380),(323,235,317,380),(400,235,240,380),(477,235,163,380),(554,235,86,380),(631,235,9,380),
        (25,305,625,315),(92,305,548,315),(169,305,471,315),(246,305,394,315),(323,305,317,315),(400,305,240,315),(465,300,158,302),(554,305,86,315),(631,305,9,315),
        (25,380,625,240),(100,388,560,248),(169,380,471,240),(246,380,394,240),(323,380,317,240),(400,380,240,240),(477,380,163,240),(554,380,86,240),(631,380,9,240),
        
        (25,455,625,165),(92,455,548,165),(169,455,471,165),(246,455,394,165),(323,455,317,165),(400,455,240,165),(477,455,163,165),(554,455,86,165),(631,455,9,165),
        (25,530,625,90),(92,530,548,90),(169,530,471,90),(246,530,394,90),(323,530,317,90),(400,530,240,90),(477,530,163,90),(554,530,86,90),(631,530,9,90),
        (25,605,625,15),(92,605,548,15),(169,605,471,15),(246,605,394,15),(323,605,317,15),(400,605,240,15),(477,605,163,15),(554,605,86,15),(631,605,9,15),
    ]
    new_image = ImageOps.crop(im,(25,20,635,615))
    new_image.show()    
    
#mejoras posibles
#entre mas chico el cuadro, mejor 
#    for coord in range(len(numbers)):
#        new_image = ImageOps.crop(im,numbers[coord])
#        new_image.save(str(coord)+'.jpg')
numbers_in_sudoku('sudoku images/sudoku.jpg')

### 4. Turn image into text
Now that we can see our numbers clearly, we are going to do some preprocessing to each image and add it into a list. 

In [4]:
import cv2
import pytesseract

def image_to_string(): 
    preprocessed_sudoku = []
    for i in range(0,81):
        im = cv2.imread(str(i) + '.jpg')
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        data = pytesseract.image_to_string(gray, config='--psm 10')
        preprocessed_sudoku.append(data)
    return(preprocessed_sudoku)
preprocessed_sudoku = image_to_string()        

ModuleNotFoundError: No module named 'cv2'

In [5]:
!pip3 install opencv-python



  Using cached opencv-python-4.4.0.40.tar.gz (88.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build opencv-python


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' 'C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\DAVIDA~1\AppData\Local\Temp\tmpvqzk7m7n'
       cwd: C:\Users\David Arreola\AppData\Local\Temp\pip-install-3s1iik34\opencv-python
  Complete output (153 lines):
  Not searching for unused variables given on the command line.
  CMake Error: CMake was unable to find a build program corresponding to "Ninja".  CMAKE_MAKE_PROGRAM is not set.  You probably need to select a different build tool.
  -- Configuring incomplete, errors occurred!
  See also "C:/Users/David Arreola/AppData/Local/Temp/pip-install-3s1iik34/opencv-python/_cmake_test_compile/build/CMakeFiles/CMakeOutput.log".
  Not searching for unused variables given on the command line.
  CMake Error at CMakeLists.txt:2 (PROJECT):
    Generator
  
      Visual Studio 15 2017 Win64
  
    could not find any instance of Visual Studio.
  
  

In [187]:
print(preprocessed_sudoku)

['4', '1', 'a', 'a', '|', 'a', 'a', 'a', 'a', '6', '', '2', '', '|', '', '', '5', '8', '7', '3', '', '1', '|', '8', '', '', '', '', 'Oo', 'Oo', '2', '4', 'Oo', 'Oo', '1', '6', '3', 'a', '6', '8', '|', '1', '7', 'a', '5', '8', '2', 'a', 'a', '6', '5', 'a', 'a', 'a', '', 'a', 'a', '9', '|', '6', 'a', '8', '4', '1', '6', 'a', 'a', '|', 'a', '3', 'a', '2', '', 'a', 'a', 'a', '|', 'a', 'a', '9', '1']


### Cleaning the list
We have to do some cleaning. This is what we are going to do:
1. First, we will remove all the weird things in our list and put them as a 0. Our model was able to predict (almost) all of the pictures correctly, the rest is trash.
2. After that, we will turn our list into a list of lists, this will help us to get our rows.
3. Lastly, we will turn each of the elemnts in our list into an integer. This will help us to avoid problems in the future with data types.
4. We are also fixing some photos we are having some problems with. This should be fixed later on.

In [188]:
#remove everything wrong

def clean_sudoku(sudoku):
    clean_sudoku = []
    for square in sudoku:
        alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 
                    'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '-', ';','', '}',
                    '{', '=', '>', '<', '.', ':', '/', ')', ']', '"', 'Z']
        
        if square in alphabet or len(square) > 1:
            clean = '0'
            clean_sudoku.append(clean)
        else:
            clean_sudoku.append(square)
    
    sudoku_list = [clean_sudoku[i:i+9] for i in range(0, len(clean_sudoku), 9)]

    return sudoku_list
    

sudoku_string = clean_sudoku(preprocessed_sudoku)
# temp = sudoku_string[2][3] 
# sudoku_string[2][3] = sudoku_string[2][4] 
# sudoku_string[2][4] = temp
# sudoku_string[2][8] = '0'

def int_to_string(sudoku):
    for i in range(len(sudoku)):
        for j in range(len(sudoku[0])):
            sudoku[i][j] = int(sudoku[i][j])

int_to_string(sudoku_string)
sudoku = sudoku_string

for l in sudoku:
    print(l)
    
    




#3 0 6 8 0 1 7 0 5
#8 2 0 0 6 5 0 0 0



[4, 1, 0, 0, 0, 0, 0, 0, 0]
[6, 0, 2, 0, 0, 0, 0, 5, 8]
[7, 3, 0, 1, 0, 8, 0, 0, 0]
[0, 0, 0, 2, 4, 0, 0, 1, 6]
[3, 0, 6, 8, 0, 1, 7, 0, 5]
[8, 2, 0, 0, 6, 5, 0, 0, 0]
[0, 0, 0, 9, 0, 6, 0, 8, 4]
[1, 6, 0, 0, 0, 0, 3, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 9, 1]


### Algorithm

Time for the algorithm. This are the following functions we will be using for our algorithm.

1. sudoku_board(sudoku)
    1. Receive the sudoku as a parameter. Prints out the sudoku
2. find_zero(sudoku)
    1. Receive the sudoku as a parameter. Finds the next zero in the sudoku. Returns the coordinates in a tuple.
3. check_sudoku(sudoku,pos,num)
    1. Receive the sudoku, the position of the number to be checked and the number being checked as parameters. It will check if the number can be placed in that box.

In [206]:
import copy
test_sudoku = copy.deepcopy(sudoku)

In [207]:
def sudoku_board(sudoku):
    for i in range(len(sudoku)):
        if i % 3 == 0 and i != 0:
            print('-------------------')
        for j in range(0,9):
            if j % 3 == 0 and j != 0:
                print('|', end = "")
            
            if j == 8:
                print(sudoku[i][j])
            else:
                print(sudoku[i][j], end = " ")
            
def find_zero(sudoku):
    for i in range(len(sudoku)):
        for j in range(len(sudoku[0])):
            if sudoku[i][j] == 0:
                return i,j
    return False

def check_sudoku(sudoku, pos, num):
    #row
    for i in range(len(sudoku[0])):
        if sudoku[pos[0]][i] == num:
            return False

    
    #column
    for i in range(len(sudoku)):
        if sudoku[i][pos[1]] == num:
            return False

    
    #box    
    box_row = pos[0]
    box_col = pos[1] 
            
    if box_row % 3 == 0:
        box_row = box_row
    elif box_row % 3 == 1:
        box_row = box_row - 1
    elif box_row % 3 == 2:
        box_row = box_row - 2

    if box_col % 3 == 0:
        box_col = box_col
    elif box_col % 3 == 1:
        box_col = box_col - 1
    elif box_col % 3 == 2:
        box_col = box_col - 2


    for i in range(box_row, box_row + 3):
        for j in range(box_col, box_col + 3):
            if sudoku[i][j] == num and (i,j) != pos:
                return False
    return True
            
def backtracking(sudoku):
    current_zero = find_zero(sudoku)
    
    if current_zero == False:
        return True
    else:
        row = current_zero[0]
        col = current_zero[1]
        
    for i in range(1,10):
        if check_sudoku(sudoku, (row,col), i):
            sudoku[row][col] = i
            
            if backtracking(sudoku):
                return True
            
            sudoku[row][col] = 0
            
    return False
    
    return sudoku

#Find a way to make a recursion and saving the last coordinate. 


In [208]:
print(sudoku_board(test_sudoku))
backtracking(test_sudoku)
print(sudoku_board(test_sudoku))


4 1 0 |0 0 0 |0 0 0
6 0 2 |0 0 0 |0 5 8
7 3 0 |1 0 8 |0 0 0
-------------------
0 0 0 |2 4 0 |0 1 6
3 0 6 |8 0 1 |7 0 5
8 2 0 |0 6 5 |0 0 0
-------------------
0 0 0 |9 0 6 |0 8 4
1 6 0 |0 0 0 |3 0 2
0 0 0 |0 0 0 |0 9 1
None
4 1 8 |6 5 9 |2 3 7
6 9 2 |4 3 7 |1 5 8
7 3 5 |1 2 8 |4 6 9
-------------------
9 5 7 |2 4 3 |8 1 6
3 4 6 |8 9 1 |7 2 5
8 2 1 |7 6 5 |9 4 3
-------------------
2 7 3 |9 1 6 |5 8 4
1 6 9 |5 8 4 |3 7 2
5 8 4 |3 7 2 |6 9 1
None


In [ ]:
# 1. Find the next zero.
# 2. Start adding one and use function: check_sudoku to make sure it fits
#    i.  If it fits, then leave it and go to the next zero and go to step 3.
#    ii. If it doesn't fit, then go to the previous zero and go to step 1. 
# 
#

In [ ]:
# 6 _ 8 _ _ 2 _ 4 _        
# 7 _ 2 _ _ _ _ 1 8        
# 1 _ _ _ 7 4 2 _ _        
# _ _ 7 4 8 _ 5 _ _     
# _ 8 _ _ 3 _ _ 9 _    
# _ _ 6 _ 2 1 4 _ _     
# _ _ 3 1 6 _ _ _ 4    
# 4 7 _ _ _ _ 6 _ 5    
# _ 6 _ 3 _ _ 1 _ 9    

In [ ]:

# 6, _, 8, _, _, 2, _, 4, _    GOOD    
# 7, _, 2, _, _, _, _, 1, 8    GOOD    
# 1, _, _, _, 7, 4, 2, _, _    ????    
# _, _, 7, 4, 8, _, 5, _, _    GOOD 
# _, 8, _, _, 3, _, _, 9, _    GOOD
# _, _, 6, _, 2, 1, 4, _, _    GOOD 
# _, _, 3, 1, 6, _, _, _, 4    GOOD
# 4, 7, _, _, _, _, 6, _, 5    GOOD
# _, 6, _, 3, _, _, 1, _, 9    GOOD

Works until there is no answer found
```
 def backtracking(sudoku):
     coord = find_zero(sudoku)
     for i in range(len(sudoku)):
         for j in range(len(sudoku[0])):
             for possible in range(1,10):
                 coord = find_zero(sudoku)
                 if check_sudoku(sudoku, coord, i) == None:
                     sudoku[coord[0]][coord[1]] = i
            
     return sudoku
```

In [ ]:
print(sudoku)